# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new norfolk,-42.7826,147.0587,20.95,48,19,3.69,AU,1730168682
1,1,adamstown,-25.0660,-130.1015,21.24,74,9,3.56,PN,1730168682
2,2,atafu village,-8.5421,-172.5159,28.20,73,94,2.80,TK,1730168682
3,3,cole harbour,44.6724,-63.4751,1.66,76,6,0.89,CA,1730168682
4,4,howard springs,-12.4970,131.0470,34.95,59,28,3.99,AU,1730168683


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [58]:
%%capture --no-display

# Create the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    # Configure the map plot
    geo="True", # show geo data
    tiles="OSM",  # OpenStreetMap tiles
    frame_width=700,
    frame_height=500,
    size='Humidity',  # Size of the points determined by Humidity
    color="City",  # Color by city
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [102]:
# Narrow down cities that fit criteria and drop any results with null values
# I am going to seek out cold places that have little wind and fairly clear skies
# Filter for temp between -15 and 5 degrees C.
filter1 = city_data_df.loc[(city_data_df['Max Temp'] > -15) & (city_data_df['Max Temp']< 5)]
# print(filter1)
# Filter for wind below 5
filter2 = filter1.loc[filter1['Wind Speed'] < 5]
# print(filter2)
# Filter for cloudiness below 25
filter3 = filter2.loc[filter2['Cloudiness'] < 25]
# print(filter3)
# Drop any rows with null values
clean_filtered = filter3.dropna().reset_index()

# Display sample data
clean_filtered

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,3,3,cole harbour,44.6724,-63.4751,1.66,76,6,0.89,CA,1730168682
1,15,15,albany,42.6001,-73.9662,3.52,68,22,0.20,US,1730168684
2,74,74,smithers,54.7804,-127.1743,2.18,93,20,1.03,CA,1730168689
3,86,86,anadyr,64.7500,177.4833,-4.72,79,20,4.00,RU,1730168690
4,236,236,sortavala,61.7053,30.6918,4.03,80,19,4.35,RU,1730168705
5,316,316,tashtyp,52.7939,89.8983,2.73,75,8,1.74,RU,1730168712
6,326,326,nagqu,31.5000,92.0000,2.71,21,0,1.61,CN,1730168713
7,328,328,dzhalagash,45.0833,64.6667,3.38,85,23,2.21,KZ,1730168713
8,335,335,suomussalmi,64.8869,28.9078,0.19,82,12,4.34,FI,1730168714
9,434,434,swift current,50.2834,-107.8014,4.45,60,0,3.09,CA,1730168724


### Step 3: Create a new DataFrame called `hotel_df`.

In [103]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_filtered[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,cole harbour,CA,44.6724,-63.4751,76,
1,albany,US,42.6001,-73.9662,68,
2,smithers,CA,54.7804,-127.1743,93,
3,anadyr,RU,64.7500,177.4833,79,
4,sortavala,RU,61.7053,30.6918,80,
5,tashtyp,RU,52.7939,89.8983,75,
6,nagqu,CN,31.5000,92.0000,21,
7,dzhalagash,KZ,45.0833,64.6667,85,
8,suomussalmi,FI,64.8869,28.9078,82,
9,swift current,CA,50.2834,-107.8014,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [104]:

# Set parameters to search for a hotel
limit = 1
radius_meters = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    # print(latitude)
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius_meters}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cole harbour - nearest hotel: Hearthstone Inn Boutique Hotel Dartmouth/Halifax
albany - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
anadyr - nearest hotel: Гостевой дом
sortavala - nearest hotel: Скандинавия
tashtyp - nearest hotel: No hotel found
nagqu - nearest hotel: Naqu Hotel
dzhalagash - nearest hotel: No hotel found
suomussalmi - nearest hotel: Kiannon kuohut
swift current - nearest hotel: York Hotel & Bar
ellsworth - nearest hotel: The Sanctuary B&B
suolahti - nearest hotel: Hotelli-Ravintola Keitele


,City,Country,Lat,Lng,Humidity,Hotel Name
0,cole harbour,CA,44.6724,-63.4751,76,Hearthstone Inn Boutique Hotel Dartmouth/Halifax
1,albany,US,42.6001,-73.9662,68,No hotel found
2,smithers,CA,54.7804,-127.1743,93,Sunshine Inn Hotel
3,anadyr,RU,64.7500,177.4833,79,Гостевой дом
4,sortavala,RU,61.7053,30.6918,80,Скандинавия
5,tashtyp,RU,52.7939,89.8983,75,No hotel found
6,nagqu,CN,31.5000,92.0000,21,Naqu Hotel
7,dzhalagash,KZ,45.0833,64.6667,85,No hotel found
8,suomussalmi,FI,64.8869,28.9078,82,Kiannon kuohut
9,swift current,CA,50.2834,-107.8014,60,York Hotel & Bar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [105]:
# Remove cities without a hotel
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != 'No hotel found'].reset_index().drop(columns='index')
hotel_df    

,City,Country,Lat,Lng,Humidity,Hotel Name
0,cole harbour,CA,44.6724,-63.4751,76,Hearthstone Inn Boutique Hotel Dartmouth/Halifax
1,smithers,CA,54.7804,-127.1743,93,Sunshine Inn Hotel
2,anadyr,RU,64.7500,177.4833,79,Гостевой дом
3,sortavala,RU,61.7053,30.6918,80,Скандинавия
4,nagqu,CN,31.5000,92.0000,21,Naqu Hotel
5,suomussalmi,FI,64.8869,28.9078,82,Kiannon kuohut
6,swift current,CA,50.2834,-107.8014,60,York Hotel & Bar
7,ellsworth,US,44.5434,-68.4195,67,The Sanctuary B&B
8,suolahti,FI,62.5642,25.8536,82,Hotelli-Ravintola Keitele


In [106]:
%%capture --no-display

# Create the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    # Configure the map plot
    geo="True", # show geo data
    tiles="OSM",  # OpenStreetMap tiles
    frame_width=700,
    frame_height=500,
    size='Humidity',  # Size of the points determined by Humidity
    color="City",  # Color by city
    scale=1.5, # the humidity is low in these cities, so I increased the scale of the points
    hover_cols=['Hotel Name', 'Country'] # added hotel names and country code to the hover
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)